In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [94]:
import json

with open('topic_testing/se_demo_test.txt', 'rb') as f:
    request = json.load(f)

In [95]:
import sys 
sys.path.append("../../../ai-engine/pkg/")
import networkx as nx
import community
import math
import logging
from log.logger import setup_server_logger
logger = logging.getLogger()
setup_server_logger(debug=False)

In [44]:
from numpy import dot
from numpy.linalg import norm
from boto3 import client as boto3_client
import json
from botocore.client import Config
import numpy as np
from copy import deepcopy

config = Config(connect_timeout=240, read_timeout=240, retries={'max_attempts': 0}, )
lambda_client = boto3_client('lambda', config=config)

def preprocess_text(text):
    mod_texts_unfiltered = tp.preprocess(text, stop_words=False, remove_punct=False)
    mod_texts = []

    for index, sent in enumerate(mod_texts_unfiltered):
        if len(sent.split(' ')) > 250:
            length = len(sent.split(' '))
            split1 = ' '.join([i for i in sent.split(' ')[:round(length / 2)]])
            split2 = ' '.join([i for i in sent.split(' ')[round(length / 2):]])
            mod_texts.append(split1)
            mod_texts.append(split2)
            continue

        if len(sent.split(' ')) <= 6:
            continue

        mod_texts.append(sent)
    return mod_texts


def cosine(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

def get_feature_vector(input_list, lambda_function):
 # logger.info("computing feature vector", extra={"msg": "getting feature vector from mind service"})
    batches_count = 300
    feature_vector = []
    count = math.ceil(len(input_list)/batches_count)
    #logger.info("computing in batches", extra={"batches count": count, "number of sentences": len(input_list)})
    for itr in range(count):
        extra_input = deepcopy(input_list[itr*batches_count:(itr+1)*batches_count])
        mind_input = json.dumps({"text": extra_input})
        mind_input = json.dumps({"body": mind_input})
        #logger.info("getting feature vector from mind service", extra={"iteration count:": itr})
        invoke_response = lambda_client.invoke(FunctionName=lambda_function, InvocationType='RequestResponse', Payload=mind_input)
        #logger.info("Request Sent", extra={"iteration count": itr})
        # logger.info("computing feature vector", extra={"msg": "Request Sent"})
        out_json = invoke_response['Payload'].read().decode('utf8').replace("'", '"')
        #print ("Recieved Response. Checking if it's valid.")
        data = json.loads(json.loads(out_json)['body'])
        #print ("Valid Response from mind service.")
        response = json.loads(out_json)['statusCode']

        if response == 200:
            feature_vector.extend(data['sent_feats'][0])
            #logger.info("Response Recieved")

            # logger.info("computing feature vector", extra={"msg": "Response Recieved"})
        #else:
            #logger.error("Invalid response from  mind service")
            # logger.error("computing feature vector", extra={"msg": "Invalid response from  mind service"})
    return feature_vector

def compute_feature_vector_gpt(segments, lambda_function):
        graph_list = {}
        input_list = []
        fv = {}
        index = 0
        for segment in segments:
            for sent in segment['ptext']:
                if sent != '':
                    input_list.append(sent)
                    
        transcript_score = get_feature_vector(input_list, lambda_function)
        for segment in segments:
            for sent in segment['ptext']:
                if sent != '':
                    graph_list[index] = (sent, segment['startTime'], segment['id'])
                    fv[index] = transcript_score[index]
                    index += 1
        assert(len(input_list)==len(fv.keys()))
        return fv, graph_list
    

def compute_feature_vector_gpt_para(sentences, lambda_function):
        graph_list = {}
        input_list = []
        fv = {}
        index = 0
        transcript_score = []
        for index, segment in enumerate(sentences):
            input_list = []
            for sent in segment:
                if sent != '':
                    input_list.append(sent)
            if input_list!=[]:        
                transcript_score=get_feature_vector(input_list, lambda_function)
                fv[index] = np.mean(transcript_score, axis=0)
                graph_list[index] = input_list
        print (len(fv.keys()))
        #assert(len(input_list)==len(fv.keys()))
        return fv, graph_list
    

In [96]:
import sys
sys.path.append("../../../ai-engine/pkg/")
import text_preprocessing.preprocess as tp
seg_list = []
for seg in request['body']["segments"]:
    seg_list.append(seg)

In [97]:
import hashlib
sent_list = []
segments = list(map(lambda x: preprocess_text(x['originalText']), seg_list))
seg_org = []
for index, seg in enumerate(seg_list):
    seg_org.append(seg)
    seg_org[-1]['ptext'] = segments[index]
segments_text = []
sent_to_id = {}
for i in segments:
    segments_text.append(i)
    
segments_map = {}
for segm in seg_org:
    if segm['ptext'] != []:
        segments_map[segm['id']] = deepcopy(segm)
index = 0
segments_order = {}
for ele in sorted(segments_map.items(), key=lambda x: x[1]['startTime'], reverse=False):
    segments_order[ele[0]] = index
    index+=1
sentence_order = {}
index = 0
for ele in sorted(segments_map.items(), key=lambda x:x[1]['startTime'], reverse=False):
    if ele[1]['ptext'] != []:
        for sent in ele[1]['ptext']:
            hash_obj = hashlib.md5(sent.encode())
            sentence_order[hash_obj.hexdigest()] = index
            index +=1

In [98]:
#lambda_function = "mind-" + request["body"]["mindId"].lower()
lambda_function = "mind-" + "01DAAQY88QZB19JQZ5PRJFR76Y".lower()
fv, graph_list = compute_feature_vector_gpt(seg_org, lambda_function)

In [92]:
def construct_graph(fv, graph_list, segments_order, sentence_order):
        meeting_graph = nx.Graph()
        yetto_prune = []
        c_weight = 0
        for nodea in graph_list.keys():
            for nodeb in graph_list.keys():
                nodea_hash = hashlib.md5(graph_list[nodea][0].encode()).hexdigest()
                nodeb_hash = hashlib.md5(graph_list[nodeb][0].encode()).hexdigest()
                if sentence_order[nodea_hash] - sentence_order[nodeb_hash] == (1):
                    print (graph_list[nodea][-1], " ==== ", graph_list[nodeb][-1])
                    c_weight = cosine(fv[nodea], fv[nodeb])
                    meeting_graph.add_edge(nodea, nodeb, weight=c_weight)
                    yetto_prune.append((nodea, nodeb, c_weight))
                    
#         X = nx.to_numpy_array(meeting_graph)
        
#         for i in range(len(X)):
#             X[i][i]=X[i].mean() 
#         norm_mat = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
        
#         meeting_graph = nx.from_numpy_array(norm_mat)
#         # logger.info("Completed Normalization", extra={"nodes: ":meeting_graph.number_of_nodes(), "edges: ": meeting_graph.number_of_edges()})
#         meeting_graph.remove_edges_from(list(map(lambda x: (x,x), range(meeting_graph.number_of_nodes()))))
#         # logger.info("Completed Normalization and after removing diagonal values", extra={"nodes: ":meeting_graph.number_of_nodes(), "edges: ": meeting_graph.number_of_edges()})
#         for i,(nodea,nodeb,score) in enumerate(yetto_prune):
#             yetto_prune[i] = (nodea,nodeb,norm_mat[nodea,nodeb])
        
        return meeting_graph, yetto_prune

def prune_edges_outlier(meeting_graph, graph_list, yetto_prune, v):
    meeting_graph_pruned = nx.Graph()
    weights = []
    for nodea, nodeb, weight in meeting_graph.edges.data():
        meeting_graph_pruned.add_nodes_from([nodea, nodeb])
        weights.append(weight["weight"])

    # q1 = np.percentile(weights, 25)
    # iqr = np.subtract(*np.percentile(weights, [75, 25]))
    # outlier = q1 - 1.5 * iqr
    q3 = np.percentile(weights, v)
    # logger.info("Outlier Score", extra={"outlier threshold is : ": outlier})
    # logger.info("Outlier Score", extra={"outlier threshold is : ": q3})
    removed = 0
    total = 0
    for indexa, indexb, c_score in meeting_graph.edges.data():
        total+=1
        if c_score["weight"]>=q3:
            removed+=1
            meeting_graph_pruned.add_edge(indexa, indexb, weight=c_score["weight"])
    #print(total-removed, removed)
    return meeting_graph_pruned


def compute_louvian_community(meeting_graph_pruned):
    # community_set = community.best_partition(meeting_graph_pruned)
    community_set = community.best_partition(meeting_graph_pruned)
    modularity_score = community.modularity(community_set, meeting_graph_pruned)
    print ("modularity score: ", modularity_score)
    community_set_sorted = sorted(community_set.items(), key=lambda kv: kv[1], reverse=False)
    return community_set_sorted


In [86]:
from copy import deepcopy
def h_communities(fv, graph_list,segments_order,sentence_order, v, h_flag):
    meeting_graph, yetto_prune = construct_graph(fv, graph_list, segments_order, sentence_order)
    original_graph = deepcopy(meeting_graph)
    meeting_graph_pruned = prune_edges_outlier(meeting_graph, graph_list, yetto_prune, v)
    community_set_sorted = compute_louvian_community(meeting_graph_pruned)
    clusters = []
    temp = []
    prev_com = 0
    for index,(word,cluster) in enumerate(community_set_sorted):
        if prev_com==cluster:
            temp.append(word)
            if index==len(community_set_sorted)-1:
                clusters.append(temp)
        else:
            clusters.append(temp)
            temp = []
            prev_com = cluster
            temp.append(word)
    if (h_flag):
        community_set_collection = []
        old_cluster = []
        # print (clusters)
        for cluster in clusters:
            if len(cluster) >= 2:
                graph_list_pruned = deepcopy(graph_list)
                for k in graph_list.keys():
                    if k not in cluster:
                        del graph_list_pruned[k]

                meeting_graph, yetto_prune = construct_graph(fv, graph_list_pruned)
                meeting_graph_pruned = prune_edges_outlier(meeting_graph, graph_list_pruned, yetto_prune, v)
                community_set = community.best_partition(meeting_graph_pruned)
                community_set_sorted = sorted(community_set.items(), key=lambda kv: kv[1], reverse=False)
                #print (community_set_sorted)
                i = 0
                prev_cluster = 9999999999999999
                for (sent, cls) in community_set_sorted:
                    if cls not in old_cluster:
                        community_set_collection.append((sent, cls))
                        old_cluster.append(cls)
                        prev_cluster = cls
                        i = cls
                    else:
                        if cls == prev_cluster:
                            community_set_collection.append((sent, i))
                            continue
                        while i in old_cluster:
                            i += 1
                        prev_cluster = cls
                        community_set_collection.append((sent, i))
                        old_cluster.append(i)
                for (sent, cls) in community_set_sorted:
                    old_cluster.append(cls)
            else:
                i = 0
                while i in old_cluster:
                    i += 1
                community_set_collection.append((cluster[0], i))
                old_cluster.append(i)
            # print(community_set_collection)
        result = []
        temp = []
        prev = 0
        for sent, cls in community_set_collection:
            if prev != cls:
                result.append(temp)
                temp = []
                #print ("cluster -=======> ", cls)
                prev = cls
            else:
                temp.append(graph_list[sent])
            #print (graph_list[sent])
        community_set_collection = sorted(community_set_collection, key = lambda x: x[1], reverse=False)

    else:
        community_set_collection = deepcopy(community_set_sorted)
        result = []
        temp = []
        prev = 0
        for sent, cls in community_set_collection:
            if prev != cls:
                result.append(temp)
                temp = []
                #print ("cluster -=======> ", cls)
                prev = cls
            else:
                temp.append(graph_list[sent][0])
            #print (graph_list[sent])
        result.append(temp)
        community_set_collection = sorted(community_set_collection, key = lambda x: x[1], reverse=False)
    return community_set_collection, meeting_graph_pruned

In [60]:
# current = 0
# segtocls = {}
# print ("--------------cluster " + str(0) + "------------ \n ")
# for word, cluster in community_set_collection:
#     segtocls[word] = cluster
#     if cluster!=current:
#         print ("--------------cluster " + str(cluster) + "------------ \n ")
#         print (graph_list[word])
#         current=cluster
#     else:
#         print (graph_list[word])

In [61]:
def get_clusters(community_set_sorted):
    clusters = []
    temp = []
    prev_com = 0
    for index,(word,cluster) in enumerate(community_set_sorted):
        if prev_com==cluster:
            temp.append(word)
            #print (temp)
            if index==len(community_set_sorted)-1:
                clusters.append(temp)
        else:
            clusters.append(temp)
            temp = []
            prev_com = cluster
            temp.append(word)
    return (clusters)

In [28]:
from collections import Counter
import statistics as st

# validate
def validate(clusters):
    validation_list = []

    for cluster in clusters:
        temp_vl = []
        for sent in cluster:
            temp_vl.append(sent_to_id[graph_list[sent]])
        validation_list.append(temp_vl)
    
    common = { }
    accuracy = []
    total_count = {}
    for val in validation_list:
        for tag in val:
            if tag in total_count.keys():
                total_count[tag] +=1
            else:
                total_count[tag] = 1
    
    for val in validation_list:
        if Counter(val).most_common()[0][0] in common.keys():
            if Counter(val).most_common()[0][1] > common[Counter(val).most_common()[0][0]]:
                common[Counter(val).most_common()[0][0]] = Counter(val).most_common()[0][1]
            else:
                continue
        else:
            common[Counter(val).most_common()[0][0]] = Counter(val).most_common()[0][1]
    
    for tag in range(0,4):
        if tag in common.keys():
            accuracy.append(common[tag]/total_count[tag])
        else:
            accuracy.append(0)
    return st.mean(accuracy)*100
# print ("accuracy is: ", st.mean(accuracy)*100)

In [29]:
for v in range(0, 100, 5):
    community_set_collection, meeting_graph_pruned = h_communities(fv, graph_list, segments_order, v, h_flag=0)
    clusters = get_clusters(community_set_collection)
    accuracy = validate(clusters)
    print (v, accuracy)

modularity score:  0.0034898741859101245


NameError: name 'part' is not defined

In [89]:
segments_order

{'eb7a2fb2248b4e4fb5304d15d792d603': 0,
 '17111370f99b498d9433149113cb7993': 1,
 'e2a1b1adc6084571b77ca56642c494cf': 2,
 '9dc22eab3121403a953e8b15ddc69530': 3,
 '1fda363d93964878b2b6c7c1960cc40e': 4,
 '5d993ad82ac346c6817b808822ee9f32': 5,
 '54b145f2e65249e8adc7b29783f28b19': 6,
 '0307c25bca384d19bf64e6d28e2d121f': 7,
 'b243d2abae394d9880561ce72254bfe7': 8,
 '44a2a0aa7bb340f0a28f4958375985c5': 9,
 '1e371cd6105a439cbf6fc4c634d50cef': 10,
 '60d843935cfd4f00b28bab790bb90d30': 11,
 '26b3780c90bb48c089177b5467527f12': 12,
 'b6b5babde859484b838508eabd4a890f': 13,
 '05e3411766fd4cf8b9514ec3765811ee': 14,
 '59992e052d6f4ff080eb8e8913b33956': 15,
 '8442631bc35845058d364bbbe4301442': 16,
 'd212ff8837a649e4b45628bf6ee55f93': 17,
 'ddf505554c88403a92465fb5a4b091ba': 18,
 'ed3186a2de7b48e5ba4d3a048aab8c7e': 19,
 '1e5abf7282b441dbaedd63c6bab8c6d5': 20}

In [99]:
community_set_collection, meeting_graph_pruned = h_communities(fv, graph_list, segments_order, sentence_order, v=0, h_flag=0)
current = 0
segtocls = {}
print ("--------------cluster " + str(0) + "------------ \n ")
for word, cluster in community_set_collection:
    segtocls[word] = cluster
    if cluster!=current:
        print ("--------------cluster " + str(cluster) + "------------ \n ")
        print (graph_list[word])
        current=cluster
    else:
        print (graph_list[word])
#clusters = get_clusters(community_set_collection)
#accuracy = validate(clusters)
#print (accuracy)
#print (clusters)

2a92aaca3087453a8e3fcb36ba0617e4  ====  28f3cab2712c4d47adca732c1ba9e9dd
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
2a92aaca3087453a8e3fcb36ba0617e4  ====  2a92aaca3087453a8e3fcb36ba0617e4
7ecdc8ab1b25466389b284328432e5fe  ====  2a92aaca3087453a8e3fcb36ba0617e4
7ecdc8ab1b25466389b284328432e5fe  ====  7ecdc8ab1b25466389b284328432e5fe
7ecdc8ab1b25466389b284328432e5fe  ====  7ecdc8ab1b25466389b284328432e5fe
7ecdc8ab1b25466389b284328432e5fe  ====  7ecdc8ab1b2

In [15]:
graph_list

{0: 'Docker is a tool designed to make it easier to create, deploy, and run applications by using containers.',
 1: 'Containers allow a developer to package up an application with all of the parts it needs, such as libraries and other dependencies, and ship it all out as one package.',
 2: 'By doing so, thanks to the container, the developer can rest assured that the application will run on any other Linux machine regardless of any customized settings that machine might have that could differ from the machine used for writing and testing the code.',
 3: 'In a way, Docker is a bit like a virtual machine.',
 4: 'But unlike a virtual machine, rather than creating a whole virtual operating system, Docker allows applications to use the same Linux kernel as the system that they are running on and only requires applications be shipped with things not already running on the host computer.',
 5: 'This gives a significant performance boost and reduces the size of the application.',
 6: 'The eart

# hierarchichal community

In [21]:
partition, community_set_collection, meeting_graph_pruned = h_communities(fv, graph_list, v=0, h_flag=0)

modularity score:  0.00835712846013996


In [31]:
seg_list = []
for seg in request["body"]["segments"]:
    seg_list.append((seg["originalText"], seg["startTime"]))
seg_list = sorted(seg_list, key=lambda kv:kv[1], reverse=False)

for seg in seg_list:
    print ("Segment: ", seg[0])
    print ("\n\n")

Segment:  Docker is a tool designed to make it easier to create, deploy, and run applications by using containers. Containers allow a developer to package up an application with all of the parts it needs, such as libraries and other dependencies, and ship it all out as one package. By doing so, thanks to the container, the developer can rest assured that the application will run on any other Linux machine regardless of any customized settings that machine might have that could differ from the machine used for writing and testing the code. In a way, Docker is a bit like a virtual machine. But unlike a virtual machine, rather than creating a whole virtual operating system, Docker allows applications to use the same Linux kernel as the system that they're running on and only requires applications be shipped with things not already running on the host computer. This gives a significant performance boost and reduces the size of the application. 



Segment:  Oh my God snaps. The earth's cli

In [48]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 72, "module": "scorer", "batches count": 1, "number of sentences": 30, "ts": "2019-11-04T10:39:41.565889Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 72, "module": "scorer", "batches count": 1, "number of sentences": 30, "ts": "2019-11-04T10:39:41.567471Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 77, "module": "scorer", "iteration count:": 0, "ts": "2019-11-04T10:39:41.569619Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 77, "module": "scorer", "iteration count:": 0, "ts": "2019-11-04T10:39:41.571126Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 79, "module": "scorer", "iteration count": 0, "ts": "2019-11-04T10:39:51.293163Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 79, "module": "scorer", "ite

{"level": "info", "filename": "grouper_segments.py", "lineno": 692, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Docker is a tool designed to make it easier to create, deploy, and run applications by using containers. Containers allow a developer to package up an application with all of the parts it needs, such as libraries and other dependencies, and ship it all out as one package. By doing so, thanks to the container, the developer can rest assured that the application will run on any other Linux machine regardless of any customized settings that machine might have that could differ from the machine used for writing and testing the code. In a way, Docker is a bit like a virtual machine. But unlike a virtual machine, rather than creating a whole virtual operating system, Docker allows applications to use the same Linux kernel as the system that they're running on and only requires applications be shipped with things not already running on the host computer. This gives a

In [36]:
cluster = [(0, 'b2728803345644d5957d6e6bb43cad3e'), (1, 'b2728803345644d5957d6e6bb43cad3e'), (2, 'b2728803345644d5957d6e6bb43cad3e'), (3, 'b2728803345644d5957d6e6bb43cad3e'), (4, 'b2728803345644d5957d6e6bb43cad3e'), (5, 'b2728803345644d5957d6e6bb43cad3e'), (9, 'f991f289ba3d4edab28ac29ec061a999'), (11, 'f991f289ba3d4edab28ac29ec061a999'), (12, 'f991f289ba3d4edab28ac29ec061a999')]

from collections import Counter
count = Counter(seg for sent, seg in cluster)

In [37]:
for seg, count in count.items():
    print (seg, count)

b2728803345644d5957d6e6bb43cad3e 6
f991f289ba3d4edab28ac29ec061a999 3


In [44]:
seg_cls = {}
seg_max = {}
clusters = [[(0, 'b2728803345644d5957d6e6bb43cad3e'), (1, 'b2728803345644d5957d6e6bb43cad3e'), (2, 'b2728803345644d5957d6e6bb43cad3e'), (3, 'b2728803345644d5957d6e6bb43cad3e'), (4, 'b2728803345644d5957d6e6bb43cad3e'), (5, 'b2728803345644d5957d6e6bb43cad3e'), (9, 'f991f289ba3d4edab28ac29ec061a999'), (11, 'f991f289ba3d4edab28ac29ec061a999'), (12, 'f991f289ba3d4edab28ac29ec061a999')], [(10, 'f991f289ba3d4edab28ac29ec061a999'), (15, '9c8094cb948147c3b9d73982b6140847'), (16, '9c8094cb948147c3b9d73982b6140847'), (17, '9c8094cb948147c3b9d73982b6140847'), (18, '9c8094cb948147c3b9d73982b6140847'), (19, '9c8094cb948147c3b9d73982b6140847'), (20, '9c8094cb948147c3b9d73982b6142347'), (21, '9c8094cb948147c3b9d73982b6142347'), (22, '9c8094cb948147c3b9d73982b6142123'), (23, '9c8094cb948147c3b9d73982b6142123'), (24, '9c8094cb948147c3b9d73982b6142123')], [(13, 'a72c7df9482b43f6a4b95d29836da09f'), (14, 'a72c7df9482b43f6a4b95d29836da09f'), (25, 'afa5f7b7433544da961a57df9e4a4a92'), (26, 'afa5f7b7433544da961a57df9e4a4a92'), (27, '47eee453d6b74a408d378252712bf6f9'), (28, '47eee453d6b74a408d378252712bf6f9'), (29, '47eee453d6b74a408d378252712bf6f9')], [(6, 'b8847ae27fce4158a032b470ce27cdad'), (7, 'b8847ae27fce4158a032b470ce27cdad'), (8, 'b8847ae27fce4158a032b470ce27cdad')]]
for index, cluster in enumerate(clusters):
    seg_cls[index] = Counter( seg for sent, seg in cluster)
    print (seg_cls[index])
    seg_count = {}
    for segid, count in seg_cls[index].items():
#         if segid not in seg_count.keys():
#             seg_count[segid] = 0
        seg_count[segid] = count
    print (seg_count)
    for segid in seg_count.keys():
        if segid not in seg_max.keys():
            seg_max[seg] = (seg_count[segid], index)
        elif seg_count[seg] >= seg_max[seg][0]:
            seg_max[seg] = (seg_count[seg], index)

Counter({'b2728803345644d5957d6e6bb43cad3e': 6, 'f991f289ba3d4edab28ac29ec061a999': 3})
{'b2728803345644d5957d6e6bb43cad3e': 6, 'f991f289ba3d4edab28ac29ec061a999': 3}
Counter({'9c8094cb948147c3b9d73982b6140847': 5, '9c8094cb948147c3b9d73982b6142123': 3, '9c8094cb948147c3b9d73982b6142347': 2, 'f991f289ba3d4edab28ac29ec061a999': 1})
{'f991f289ba3d4edab28ac29ec061a999': 1, '9c8094cb948147c3b9d73982b6140847': 5, '9c8094cb948147c3b9d73982b6142347': 2, '9c8094cb948147c3b9d73982b6142123': 3}
Counter({'47eee453d6b74a408d378252712bf6f9': 3, 'a72c7df9482b43f6a4b95d29836da09f': 2, 'afa5f7b7433544da961a57df9e4a4a92': 2})
{'a72c7df9482b43f6a4b95d29836da09f': 2, 'afa5f7b7433544da961a57df9e4a4a92': 2, '47eee453d6b74a408d378252712bf6f9': 3}
Counter({'b8847ae27fce4158a032b470ce27cdad': 3})
{'b8847ae27fce4158a032b470ce27cdad': 3}


In [41]:
seg_max

{'f991f289ba3d4edab28ac29ec061a999': (1, 3)}

In [91]:

user_id_map = {}
user_id_map = {"3e1a008f734448b0ad9190778449af81":"Cullen","b4a57b25de68446cac990f856d3fe4d5":"Cullen","716067a60a1a4034abc49a12ecafb39b":"Cullen","2f506a3d9e814de69d46a1fbf949fdc9":"Cullen","8d6db5f7d9b74c54ba38fe710ffcaf3f":"Krishna Sai", "c66797a92e6d46ad9573926e57f7dac3":"Nisha Yadav","31a3ba4761854ad9a041ddf1c4c6a1dc":"Reagan Rewop","84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}

for groupid in group['group'].keys():
    user_list =[]
    seg_list = []
    keyphrase = []
    for segi in group['group'][groupid]:
        if segi['spokenBy'] not in user_list:
            user_list.append(segi['spokenBy'])
        seg_list.append(segi['originalText'])
        #keyphrase.append(get_desc(segi['originalText']))
    #print ("User", end=" ")
    print ("Group Id: ", groupid)
    print (*[user_id_map[user] for user in user_list], sep=", ", end=" ")
    print ("Discussed \n\n Text: ", *seg_list, "\n\n  ")
    #print ("Keyphrases: ", end="")
    #print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
    print ( "\n")

NameError: name 'group' is not defined

# using mind and testing it's affinity

In [76]:
import pickle

with open("se_mind.pkl","rb") as f:
    mind = pickle.load(f)

In [77]:
cluster_sent = {}
sent_cluster = {}
for group in clusters:
    for sent in group:
        closest_sent = 0
        closest_mind_score = 0
        closest_mind = None
        temp = 0
        for fv_mind_id in mind["feature_vector"].keys():
            temp = cosine(fv[sent], mind["feature_vector"][fv_mind_id])
            if temp > closest_mind_score:
                closest_mind = fv_mind_id
                closest_mind_score = temp
        if closest_mind not in cluster_sent.keys():
            cluster_sent[closest_mind] = []
        cluster_sent[closest_mind].append(sent)
        sent_cluster[sent] = closest_mind

In [78]:
for cluster_id in cluster_sent.keys():
    print ("-------- cluster chosen -------",  mind["sentence"][cluster_id], "\n\n")
    for sent in cluster_sent[cluster_id]:
        print (graph_list[sent], "\n\n\n")

-------- cluster chosen ------- In the past we shipped a container image and a helm chart still all available but deprecated that you could run and you just choose one of the compiled in cloud providers however this proved to be a huge maintenance burden especially with managing dependencies for all the clouds 


Now does not allow you to put data in the audio of or request boot days against. 



Hi hey video about thirty much better Java merchant machine. 



Weve drilled down its the maps resource and then the API resource and then the geocode resource and we have even added Jason as a resource and that is because the Google Maps API can return data in a number of different products. 



Gonna have the optional number of instances that you want to use based on the performance. 



Is gonna ask you for the version that you wanna to I am running and check and use the latest and again since it is the service based on asking search. 



So the some my this topic for the video. 



This i

In [186]:
weights = []
for nodea, nodeb , weight in meeting_graph.edges.data():
    weights.append(weight["weight"])

In [187]:
len(weights)

5778

# keyword comparison

In [51]:
comparison_list = []
for nodea, nodeb, weight in original_graph.edges.data():
    comparison_list.append((nodea, nodeb, weight["weight"]))

comparison_list = sorted(comparison_list, key=lambda kv:kv[-1], reverse=True)
for a,b,distance in comparison_list:
    print ("--------sent--------\n\n", graph_list[a],"\n\n", graph_list[b], "\n", distance, "\n", segtocls[a], "=>", segtocls[b],"\n\n\n\n")

--------sent--------

 So let us dig into that a little bit. 

 Let is take a look at it. 
 0.9553863484644594 
 1 => 1 




--------sent--------

 Right, so we cannot we just started to figure that out. 

 So like what should we do about that? 
 0.9447276795181943 
 8 => 8 




--------sent--------

 Alright, we will take this off for you. 

 Okay, so we will take that up as a backlog item. 
 0.9401306334887837 
 1 => 4 




--------sent--------

 So even that yeah, so I discussed this bit. 

 So I just took the I just want to be really yeah, it is offline. 
 0.9393572591086908 
 17 => 17 




--------sent--------

 So if you look at the schema structure, so we got an idea. 

 So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. 
 0.9333535132353137 
 0 => 0 




--------sent--------

 It became need to see if we need to prioritize this over that let us prioritize what we discussed which is you 

 0.8733829893259717 
 1 => 3 




--------sent--------

 So we replace that there are some policies. 

 So that is the tradeoff if we are operating on teams. 
 0.8733713570337718 
 10 => 8 




--------sent--------

 It became need to see if we need to prioritize this over that let us prioritize what we discussed which is you know, like you it is all of these things right? 

 You know if there is a convinced I am good with it, but just keep that in the back of your mind. 
 0.8733478644774466 
 9 => 9 




--------sent--------

 Okay, see the real the world really want to know kind of what I am trying to say is when we take up some task.Let is make sure it gets all the way to production before we get to something else right unless it is an experimental investigative thing to just develop the discipline that we stick with something and see through the production right so we do not give a reason okay. 

 I mean meeting it is to uuids now. 
 0.873193487254924 
 9 => 16 




--------sent---

 Alright, we will take this off for you. 

 we can check when you are loading the asset side. 
 0.8593602376600696 
 1 => 1 




--------sent--------

 Right, so we cannot we just started to figure that out. 

 So what we will do see the plan of action is if the SDK gets integrated if we are able to test in staging and everything works. 
 0.8593584325499378 
 8 => 0 




--------sent--------

 We can make it zero to be minimum instead of like showing that negative value. 

 So what we will do see the plan of action is if the SDK gets integrated if we are able to test in staging and everything works. 
 0.8593366892261269 
 4 => 0 




--------sent--------

 I mean they did not change.Okay, so in staging the changes are yeah, they changed I am not sure whether you have whether the board was reinvited. 

 So I just took the I just want to be really yeah, it is offline. 
 0.8593224721542161 
 16 => 17 




--------sent--------

 Thus the top few ones and if next whenever we add new relatio

 There is no restrictions because no it is not necessary really hard even sequence for generally rdbms schema means different because we try to say this is of this type and we are here in force types and trying to Signal this carbonyl Etc. 

 Let is see how it goes through and then otherwise, I am working on the summary service like basic coding is done. 
 0.8499781905145278 
 0 => 1 




--------sent--------

 No, I think it has nothing to do with it the left side if this offline so we have Google transcripts for Collins call. 

 Also connects to the city and cash thing, right? 
 0.849977729737346 
 18 => 18 




--------sent--------

 All the other types are not the same because they support decent marshalling and marshaling right? 

 Or that we already have the kind of conditions climb said other things. 
 0.8499663077771601 
 8 => 8 




--------sent--------

 Let is see how it goes through and then otherwise, I am working on the summary service like basic coding is done. 

 So lik

 3 => 10 




--------sent--------

 And I think that is all that is needed right now. 

 Let is see how it goes through and then otherwise, I am working on the summary service like basic coding is done. 
 0.8428196266100173 
 8 => 1 




--------sent--------

 I mean some of these things can come back and bite us once you have you know, like a big graph is head is the glass used in production mirror. 

 So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neoXnumberXj or Neptune and all this thing up up. 
 0.8428139615817538 
 8 => 16 




--------sent--------

 Okay, did we resolve that issue of the first time stamp being incorrect in the full transcript? 

 So it is showing up i

...

 I mean some of these things can come back and bite us once you have you know, like a big graph is head is the glass used in production mirror. 

 So even that yeah, so I discussed this bit. 
 0.8039702610132019 
 8 => 17 




--------sent--------

 Let is see how it goes through and then otherwise, I am working on the summary service like basic coding is done. 

 Because it is just the Lander services that need to be removed. 
 0.8039692618694735 
 1 => 15 




--------sent--------

 All one word is always coming wrong EXO feel sort of SFU. 

 What what how I have been testing is I just read out some random or or this.Text from the document and then explicitly insert some action items and I am just ask XnumberX to do the same things. 
 0.8039617417192969 
 18 => 17 




--------sent--------

 I am not yet showing the time I am showing the text and also, yeah, I will work on it. 

 We can make it zero to be minimum instead of like showing that negative value. 
 0.8039350296422757 
 17 

 0.8001008375743602 
 18 => 16 




--------sent--------

 If you want to speaker info, so we need to handle some formatting related things, you know. 

 An example of a schema change leader of like what is it like it just in general for girls like what what?. 
 0.800090522552994 
 3 => 10 




--------sent--------

 And one thing I have observed is just posted an MLA. 

 Like registered like like completed but invitation process yesterday. 
 0.800090474091389 
 17 => 18 




--------sent--------

 You have to break our primary database and make a data table schema cell phones representation of data now does not really need to make sure variable in graph terminology. 

 But also the you know, yeah, so right now we have everything from like at least in production, which is populated in Neptune. 
 0.8000867577053004 
 0 => 16 




--------sent--------

 They need to know because everybody uses really way to query the through recipes Etc. 

 So the CIA, you know we shall again just to rei

 8 => 9 




--------sent--------

 And then we use QuickTime or some other way to construct a custom playlist. 

 An example of a schema change leader of like what is it like it just in general for girls like what what?. 
 0.7962607479031385 
 4 => 10 




--------sent--------

 I will just deployed deployed then I start to pick the Gypsy and calls you. 

 It is just we need to remap in the production. 
 0.7962373956573539 
 18 => 4 




--------sent--------

 All the other types are not the same because they support decent marshalling and marshaling right? 

 We set it to level bonding than errors. 
 0.7962314320931203 
 8 => 4 




--------sent--------

 Alright, we will take this off for you. 

 Like should we like should deem work on that part?. 
 0.7962253107445045 
 1 => 10 




--------sent--------

 It is this small change article as I change it. 

 Oh, no, no just like watch Lingus part is done. 
 0.7962138013508742 
 1 => 16 




--------sent--------

 All the other types ar

 0.7927619751084302 
 1 => 15 




--------sent--------

 It became need to see if we need to prioritize this over that let us prioritize what we discussed which is you know, like you it is all of these things right? 

 Were in for whatever reason we decided to calculate it in Franklin and we went ahead as implementation. 
 0.792752471957674 
 9 => 13 




--------sent--------

 So it seems like a different provider where it does not support like all Rowling has is video recorded video. 

 So he will be sharing in next few minutes. 
 0.7927435152319358 
 3 => 10 




--------sent--------

 Were in for whatever reason we decided to calculate it in Franklin and we went ahead as implementation. 

 For example today we have something like to today is structure of graph will have context instance ID. 
 0.7927422870083878 
 13 => 0 




--------sent--------

 So what you have to do is that by the time the quarter started someone was already speaking. 

 Yesterday, so he said that they will h

KeyError: 160

In [30]:
temp = []
for nodea, nodeb, weight in meeting_graph_pruned.edges.data():
    temp.append(nodea)
    temp.append(nodeb)
    
sorted(temp)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [31]:
from collections import Counter

for (nodea, count ) in Counter(temp).most_common()[:4]:
    print (graph_list[nodea])

It became need to see if we need to prioritize this over that let us prioritize what we discussed which is you know, like you it is all of these things right?
Now in this case, I do not see you.
So that is what it is also generating errors.
So we did not we realize a little late on the so after that I think after that the summaries are coming up.


In [32]:
Counter(temp).most_common()

[(19, 142),
 (136, 141),
 (75, 127),
 (45, 124),
 (87, 124),
 (40, 122),
 (23, 119),
 (164, 119),
 (70, 118),
 (33, 117),
 (122, 116),
 (125, 116),
 (0, 107),
 (39, 107),
 (56, 107),
 (79, 107),
 (165, 107),
 (118, 106),
 (127, 105),
 (92, 104),
 (13, 103),
 (102, 103),
 (30, 102),
 (25, 101),
 (54, 98),
 (6, 97),
 (149, 97),
 (94, 94),
 (185, 94),
 (78, 93),
 (100, 93),
 (195, 93),
 (18, 91),
 (32, 91),
 (137, 91),
 (142, 91),
 (58, 90),
 (3, 89),
 (64, 88),
 (28, 86),
 (60, 86),
 (69, 86),
 (184, 86),
 (119, 82),
 (155, 82),
 (114, 81),
 (2, 80),
 (89, 80),
 (151, 80),
 (52, 78),
 (130, 77),
 (171, 77),
 (179, 77),
 (42, 76),
 (65, 75),
 (189, 75),
 (27, 73),
 (74, 71),
 (140, 71),
 (150, 71),
 (170, 71),
 (192, 71),
 (26, 71),
 (158, 71),
 (196, 70),
 (138, 69),
 (186, 69),
 (82, 68),
 (152, 66),
 (200, 66),
 (177, 66),
 (41, 65),
 (143, 65),
 (61, 64),
 (4, 63),
 (168, 63),
 (135, 62),
 (201, 62),
 (63, 61),
 (117, 61),
 (197, 61),
 (176, 59),
 (35, 59),
 (11, 58),
 (126, 57),
 (13

In [33]:
community_set_sorted

[(0, 0),
 (3, 0),
 (6, 0),
 (9, 0),
 (10, 0),
 (13, 0),
 (20, 0),
 (30, 0),
 (32, 0),
 (37, 0),
 (40, 0),
 (42, 0),
 (48, 0),
 (49, 0),
 (51, 0),
 (55, 0),
 (56, 0),
 (57, 0),
 (58, 0),
 (59, 0),
 (60, 0),
 (61, 0),
 (62, 0),
 (63, 0),
 (65, 0),
 (72, 0),
 (73, 0),
 (75, 0),
 (77, 0),
 (78, 0),
 (81, 0),
 (88, 0),
 (94, 0),
 (97, 0),
 (100, 0),
 (104, 0),
 (106, 0),
 (107, 0),
 (108, 0),
 (119, 0),
 (120, 0),
 (130, 0),
 (135, 0),
 (137, 0),
 (138, 0),
 (140, 0),
 (142, 0),
 (150, 0),
 (165, 0),
 (173, 0),
 (176, 0),
 (179, 0),
 (180, 0),
 (184, 0),
 (185, 0),
 (189, 0),
 (194, 0),
 (195, 0),
 (196, 0),
 (198, 0),
 (199, 0),
 (1, 1),
 (4, 1),
 (5, 1),
 (11, 1),
 (12, 1),
 (14, 1),
 (19, 1),
 (21, 1),
 (23, 1),
 (33, 1),
 (34, 1),
 (36, 1),
 (39, 1),
 (41, 1),
 (53, 1),
 (64, 1),
 (68, 1),
 (70, 1),
 (74, 1),
 (76, 1),
 (79, 1),
 (82, 1),
 (91, 1),
 (95, 1),
 (98, 1),
 (102, 1),
 (111, 1),
 (112, 1),
 (121, 1),
 (124, 1),
 (125, 1),
 (126, 1),
 (127, 1),
 (132, 1),
 (139, 1),
 (141, 1),

In [1]:
import spacy
from benepar.spacy_plugin import BeneparComponent
import benepar

benepar.download('benepar_en')
nlp = spacy.load('en')

nlp.add_pipe(BeneparComponent('benepar_en'))
doc = nlp("The time for action is now. It's never too late to do something.")
sent = list(doc.sents)[0]
print(sent._.parse_string)
# (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
print(sent._.labels)
# ('S',)
print(list(sent._.children)[0])
# The time for action

[nltk_data] Downloading package benepar_en to /home/ray__/nltk_data...
[nltk_data]   Package benepar_en is already up-to-date!



(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
('S',)
The time for action


In [4]:
doc = nlp("AWS transcribe machine is working good in staging2.")
sent = list(doc.sents)[0]
print(sent._.parse_string)
# (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
print(sent._.labels)
# ('S',)
print(list(sent._.children)[0])
# The time for action

(S (NP (NN AWS) (NN transcribe) (NN machine)) (VP (VBZ is) (VP (VBG working) (ADJP (JJ good)) (PP (IN in) (NP (NN staging2))))) (. .))
('S',)
AWS transcribe machine


In [8]:
doc = nlp("Maybe it four times at least he did send a signal.")
sent = list(doc.sents)[0]
print(sent._.parse_string)
# (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
print(sent._.labels)
# ('S',)
print(list(sent._.children)[0])
# The time for action


(S (ADVP (RB Maybe)) (NP (PRP it)) (NP (NP (CD four) (NNS times)) (IN at) (JJS least)) (NP (PRP he)) (VP (VBD did) (VP (VB send) (NP (DT a) (NN signal)))) (. .))
('S',)
Maybe


In [17]:
doc = nlp("I started seeing the code Recorder Server code on and we probably like by today I'll start working on that issue let.")
sent = list(doc.sents)[0]
print(sent._.parse_string)

(S (NP (PRP I)) (VP (VBD started) (S (VP (VBG seeing) (NP (DT the) (NN code) (NNP Recorder) (NNP Server) (VB code)) (PP (RP on))))))


In [10]:
doc = nlp("You just need to be careful of when we decide to trigger an end call ourselves.")
sent = list(doc.sents)[0]
print(sent._.parse_string)

(S (NP (PRP You)) (ADVP (RB just)) (VP (VBP need) (S (VP (TO to) (VP (VB be) (ADJP (JJ careful) (PP (IN of) (SBAR (WHADVP (WRB when)) (S (NP (PRP we)) (VP (VBP decide) (S (VP (TO to) (VP (VB trigger) (NP (DT an) (NN end) (VB call)) (NP (PRP ourselves)))))))))))))) (. .))


In [11]:
doc = nlp("Then why don't you talk about the websocket?.")
sent = list(doc.sents)[0]
print(sent._.parse_string)

(SBARQ (RB Then) (WHADVP (WRB why)) (SQ (VBP do) (RB n't) (NP (PRP you)) (VP (VB talk) (PP (IN about) (NP (DT the) (NN websocket))))) (. ?) (. .))


In [18]:
doc = nlp("We can not appear or you can follow the path.")
sent = list(doc.sents)[0]
print(sent._.parse_string)

(S (S (NP (PRP We)) (VP (MD can) (RB not) (VP (VB appear)))) (CC or) (S (NP (PRP you)) (VP (MD can) (VP (VB follow) (NP (DT the) (NN path))))) (. .))


In [13]:
doc = nlp("You know can you and we may take a look at this?")
sent = list(doc.sents)[0]
print(sent._.parse_string)

(S (S (NP (PRP You)) (VP (VBP know) (MD can) (NP (PRP you)))) (CC and) (S (NP (PRP we)) (VP (MD may) (VP (VB take) (NP (DT a) (NN look)) (PP (IN at) (NP (DT this)))))) (. ?))


In [19]:
doc = nlp("Same work is not a new industry usually frameworks aren't designed with the visa that from white of old for common functionality in industry.")
sent = list(doc.sents)[0]
print(sent._.parse_string)

(S (NP (NP (JJ Same) (NN work)) (VP (VBZ is) (RB not) (NP (DT a) (JJ new) (NN industry)))) (ADVP (RB usually)) (NP (NNS frameworks)) (VP (VBP are) (RB n't) (VP (VBN designed) (PP (IN with) (NP (NP (NP (DT the) (NN visa)) (IN that)) (PP (IN from) (ADJP (NP (JJ white)) (PP (IN of) (ADJP (JJ old))))) (PP (IN for) (NP (NP (JJ common) (NN functionality)) (PP (IN in) (NP (NN industry))))))))) (. .))


In [57]:
norm_mat.shape

(30, 30)

In [61]:
print(adj_mat)

  (0, 1)	0.8651792770213481
  (0, 2)	0.7946813537519716
  (0, 3)	0.9099471607978834
  (0, 4)	0.8976771458160738
  (0, 5)	0.7568137128144239
  (0, 6)	0.4906986211200429
  (0, 7)	0.43744628860166884
  (0, 8)	0.34147844065294175
  (0, 9)	0.9494072936956971
  (0, 10)	0.69106767277158
  (0, 11)	0.8975273103758501
  (0, 12)	0.892529830750924
  (0, 13)	0.6640106117563596
  (0, 14)	0.5747607357040782
  (0, 15)	0.6934270185454394
  (0, 16)	0.5808692868480433
  (0, 17)	0.6457705045291043
  (0, 18)	0.644525822060585
  (0, 19)	0.6576437718170873
  (0, 20)	0.644525822060585
  (0, 21)	0.6576437718170873
  (0, 22)	0.7153621034472333
  (0, 23)	0.654120008595783
  (0, 24)	0.6700808519867759
  (0, 25)	0.6418413691780865
  :	:
  (29, 4)	0.6809634156571828
  (29, 5)	0.6166618553789603
  (29, 6)	0.5083742721392881
  (29, 7)	0.5157890534296394
  (29, 8)	0.42471114025624057
  (29, 9)	0.672142840044772
  (29, 10)	0.6012548366496223
  (29, 11)	0.7231785370913997
  (29, 12)	0.67513296036664
  (29, 13)	0.6534710

In [101]:
import sys 
sys.path.append("../../../ai-engine/pkg/")
from text_preprocessing import preprocess as tp

In [108]:
tp.preprocess("I don't know whether it looks locally on.", stop_words=False, pos=True)

(['I do not know whether it looks locally on.'],
 [[('I', 'PRP'),
   ('do', 'VBP'),
   ('not', 'RB'),
   ('know', 'VB'),
   ('whether', 'IN'),
   ('it', 'PRP'),
   ('looks', 'VBZ'),
   ('locally', 'RB'),
   ('on', 'IN'),
   ('.', '.')]])

In [111]:
tp.preprocess("karthik, I have deployed it.", stop_words=True, remove_punct=False)

['karthik, I deployed it.']